<a href="https://colab.research.google.com/github/esusrutha/zeroshot/blob/main/file1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Read in the CSV files
RESULTS_DIR = 'drive/MyDrive'
df1 = pd.read_csv(RESULTS_DIR + '/D11.csv', encoding='latin1', engine='python')
df2 = pd.read_csv(RESULTS_DIR + '/D2.csv', encoding='latin1', engine='python')


# Merge the dataframes
merged_df = pd.concat([df1, df2], axis=0)

# Create a new column called 'category' and initialize it with 'None'
merged_df['category'] = None

# Create a list of adive,china,mask,news,transporation,usa and vaccine
Advice = ["['Stay at home']","['wash hands']","['wear mask']","['social distancing']"]
China = ["['Wuhan']","['China Coronavirus Updates']","['China news']","['other tweets related to China']"]
Mask = ["['Mask shortage']","['wear mask']","['mask types']","['N50']","['N95']","['3M8210']","['3M9001']","['3M9322']","['3M9501']"]
News = ["['Coronavirus updates']","['news']","['rules']"]
Transportation = ["['Flights']","['traffic']","['traveling']"]
USA = ["['U.S. Coronavirus Updates']","['COVID19']","['U.S. news']","['United States']","['US']","['USA']"]
Vaccine = ["['Vaccine news']","['vaccine progress']","['vaccine injection']"]

# Iterate over the rows of the dataframe
for index, row in merged_df.iterrows():
    # Check if the hashtags column contains  'Stay at home','wash hands','wear mask' or 'social distancing'
    if row['hashtags'] in Advice:
        # If it does, set the category to 'Advice'
        merged_df.at[index, 'category'] = 'Advice'
    elif row['hashtags'] in China:
        # If it does, set the category to 'China'
        merged_df.at[index, 'category'] = 'China'
    elif row['hashtags'] in Mask:
        # If it does, set the category to 'Mask'
        merged_df.at[index, 'category'] = 'Mask'
    elif row['hashtags'] in News:
        # If it does, set the category to 'News'
        merged_df.at[index, 'category'] = 'News'
    elif row['hashtags'] in Transportation:
        # If it does, set the category to 'Transportation'
        merged_df.at[index, 'category'] = 'Transportation'
    elif row['hashtags'] in USA:
        # If it does, set the category to 'USA'
        merged_df.at[index, 'category'] = 'USA'
    elif row['hashtags'] in Vaccine:
        # If it does, set the category to 'Vaccine'
        merged_df.at[index, 'category'] = 'Vaccine'
    
# Print the updated dataframe
print(merged_df["category"].value_counts())

Mask              469
Advice            311
USA               224
News              216
Vaccine           134
Transportation    132
China             131
Name: category, dtype: int64


In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install ConceptNet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 KB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 KB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Read the merged DataFrame containing the tweets and categories
merged_df = pd.concat([df1, df2], axis=0)

# Load the pre-trained S-BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Convert the tweets into sentence embeddings
tweet_embeddings = model.encode(merged_df['text'].values)

# Define the possible label names
label_names = ['Vaccine','USA','Transportation','News','Mask','China','Advice']

# Convert the label names into label embeddings
label_embeddings = model.encode(label_names)

# Calculate the cosine similarities between each tweet embedding and each label embedding
cos_similarities = cosine_similarity(tweet_embeddings, label_embeddings)

# Assign the label with the highest cosine similarity to each tweet
predicted_labels = [label_names[i] for i in cos_similarities.argmax(axis=1)]

# Add the predicted labels to the merged DataFrame
merged_df['predicted_label'] = predicted_labels

# Print the resulting DataFrame
print(predicted_labels)

['China', 'USA', 'Advice', 'Vaccine', 'News', 'News', 'Vaccine', 'Advice', 'Vaccine', 'USA', 'Mask', 'USA', 'Vaccine', 'USA', 'Vaccine', 'Vaccine', 'Advice', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 'Advice', 'USA', 'China', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 'Advice', 'Vaccine', 'News', 'Vaccine', 'Advice', 'Vaccine', 'News', 'China', 'Mask', 'China', 'USA', 'USA', 'Vaccine', 'Vaccine', 'Vaccine', 'USA', 'Vaccine', 'Vaccine', 'Vaccine', 'China', 'Vaccine', 'Vaccine', 'China', 'USA', 'Vaccine', 'Vaccine', 'News', 'Vaccine', 'Vaccine', 'Vaccine', 'China', 'Vaccine', 'Vaccine', 'China', 'USA', 'Advice', 'Vaccine', 'Mask', 'Vaccine', 'News', 'News', 'China', 'Advice', 'China', 'News', 'Vaccine', 'News', 'Vaccine', 'Advice', 'China', 'USA', 'News', 'News', 'Vaccine', 'Vaccine', 'Advice', 'China', 'Vaccine', 'Vaccine', 'China', 'Vaccine', 'USA', 'USA', 'Vaccine', 'Vaccine', 'USA', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 'Vaccine', 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the pre-trained S-BERT model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/sentence-transformers-all-distilroberta-v1")
model = AutoModel.from_pretrained("sentence-transformers/sentence-transformers-all-distilroberta-v1")

# Load the ConceptNet knowledge graph
\

# (You will need to download and preprocess the ConceptNet graph)
graph = load_conceptnet_graph()

# Retrofit the ConceptNet graph to include sentence and label embeddings
graph = retrofit_graph(graph, model)

# Define the set of possible labels
labels = ["Vaccine","USA","Transportation","News","Mask","China","Advice"]

# Convert the labels to embeddings
label_embeddings = []
for label in labels:
    label_embedding = model.encode(label)
    label_embeddings.append(label_embedding)

# Loop through each tweet in X
for tweet in X:
    # Convert the tweet to an embedding
    tweet_embedding = model.encode(tweet)
    
    # Project the tweet and label embeddings to the knowledge graph embedding space
    projected_tweet_embedding = torch.matmul(tweet_embedding, P)
    projected_label_embeddings = []
    for label_embedding in label_embeddings:
        projected_label_embedding = torch.matmul(label_embedding, P)
        projected_label_embeddings.append(projected_label_embedding)
        
    # Calculate the cosine similarity between the tweet embedding and each label embedding
    similarities = []
    for projected_label_embedding in projected_label_embeddings:
        similarity = torch.cosine_similarity(projected_tweet_embedding, projected_label_embedding, dim=0)
        similarities.append(similarity)
        
    # Find the label with the highest similarity to the tweet
    max_similarity = max(similarities)
    max_similarity_index = similarities.index(max_similarity)
    predicted_label = labels[max_similarity_index]
    
    # Output the predicted label for the tweet
    print("Tweet: ", tweet)
    print("Predicted label: ", predicted_label)

OSError: ignored